# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities_df = pd.read_csv('output_data/cities.csv')

cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Ushuaia,67,AR,1587954096,69,-54.80,-68.30,39.20,11.41
1,1,Mareeba,0,AU,1587954054,63,-17.00,145.43,86.00,5.99
2,2,Shingū,40,JP,1587954096,59,33.73,135.98,62.60,12.75
3,3,Punta Arenas,15,CL,1587954096,74,-53.15,-70.92,32.00,4.70
4,4,Ankazoabo,19,MG,1587954097,75,-22.28,44.52,63.52,7.58


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

In [8]:
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,dissipating=False, max_intensity=8)

# Display figure

fig = gmaps.figure(center=(11.0, -10.0), zoom_level=2)

# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# cities_of_interest selection
cities_of_interest_df = cities_df.loc[(cities_df["Max Temp"] <= 78) & 
                                    (cities_df["Max Temp"] >= 70) &
                                    (cities_df["Humidity"] <= 60) &
                                    (cities_df["Wind Speed"] <= 10.5) ]

cities_of_interest_df = cities_of_interest_df.dropna()

cities_of_interest_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
23,23,Show Low,1,US,1587954008,16,34.25,-110.03,73.40,9.17
38,38,Korla,0,CN,1587954102,12,41.76,86.15,75.13,3.80
52,52,Stephenville,1,US,1587954103,43,32.22,-98.20,71.60,6.93
122,122,Iberia,1,US,1587954109,49,29.95,-91.75,71.60,8.05
126,126,Zharkent,33,KZ,1587954109,33,44.16,80.00,73.85,7.23
146,146,Najrān,40,SA,1587954113,44,17.49,44.13,77.00,5.46
191,191,Adrar,0,MR,1587954117,17,20.50,-10.07,76.19,10.04
205,205,Alice Springs,62,AU,1587953986,44,-23.70,133.88,77.00,9.17
235,235,Langfang,0,CN,1587954121,33,39.51,116.69,71.01,4.43
237,237,Airai,29,TL,1587954121,44,-8.93,125.41,72.05,3.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

for index, row in cities_of_interest_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    # Use the nearbysearch Google Place API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    # convert to json
    name_address = name_address.json()
    
    try:
        cities_of_interest_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        pass
    
cities_of_interest_df.reset_index(drop = True)

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,23,Show Low,1,US,1587954008,16,34.25,-110.03,73.40,9.17,Hampton Inn & Suites Show Low-Pinetop
1,38,Korla,0,CN,1587954102,12,41.76,86.15,75.13,3.80,Kangcheng Jian Guo International Hotel
2,52,Stephenville,1,US,1587954103,43,32.22,-98.20,71.60,6.93,Holiday Inn Express & Suites Stephenville
3,122,Iberia,1,US,1587954109,49,29.95,-91.75,71.60,8.05,NaN
4,126,Zharkent,33,KZ,1587954109,33,44.16,80.00,73.85,7.23,гостиница Satti
5,146,Najrān,40,SA,1587954113,44,17.49,44.13,77.00,5.46,NaN
6,191,Adrar,0,MR,1587954117,17,20.50,-10.07,76.19,10.04,NaN
7,205,Alice Springs,62,AU,1587953986,44,-23.70,133.88,77.00,9.17,DoubleTree by Hilton Hotel Alice Springs
8,235,Langfang,0,CN,1587954121,33,39.51,116.69,71.01,4.43,Holiday Inn Express Langfang Park View
9,237,Airai,29,TL,1587954121,44,-8.93,125.41,72.05,3.24,NaN


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cities_of_interest_df.iterrows()]
locations_of_interest = cities_of_interest_df[["Lat", "Lng"]]

In [17]:

# Add marker layer ontop of heat map

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer and markers
heat_layer = gmaps.heatmap_layer(locations,dissipating=False, max_intensity=8)
markers = gmaps.marker_layer(locations_of_interest,info_box_content =hotel_info)

# Display figure

fig = gmaps.figure(center=(11.0, -10.0), zoom_level=2)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))